In [1]:
# Read data from all files into a single Pandas Dataframe

import pandas as pd
import os

dfs = []
for file in sorted(
        os.listdir(os.path.join("..", "data")),
        key=lambda x: (int(x[5]), int(x.split("Cycle")[-1][:-4])),
    ):
    
    path_to_file = os.path.join("..", "data", file)
    df = pd.read_csv(path_to_file)
    dfs.append(df)
    
df = pd.concat(dfs).reset_index(drop=True)

In [2]:
# Print columns/features present in the data

df.columns

Index(['Voltage [V]', 'Current [A]', 'AhStep [Ah]', 'TempBottom [C]',
       'CRate [-]', 'SOC [%]', 'Cycle [-]', 'Time [s]'],
      dtype='object')

In [3]:
# Start interactive visualization JupyterDash

from jupyter_dash import JupyterDash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [4]:
from dash import dcc, html
import plotly.express as px
import plotly.graph_objs as go

df1_1 = df[(df["Cycle [-]"] <= 5) & (df["CRate [-]"] == 1)]
df1_2 = df[(df["Cycle [-]"] <= 5) & (df["CRate [-]"] == 2)]
fig1 = go.Figure([
    go.Scatter(
        name='State-of-Charge during the first five cycles of discharging cell at CRate 1',
        x=df1_1['Time [s]'],
        y=df1_1["SOC [%]"],
        mode='markers',
        opacity=0.5,
        showlegend=True
    ),
    go.Scatter(
        name='State-of-Charge during the first five cycles of discharging cell at CRate 2',
        x=df1_2['Time [s]'],
        y=df1_2["SOC [%]"],
        mode='markers',
        opacity=0.5,
        showlegend=True
    ),
])
fig1.update_layout(yaxis_title='SOC [%]', xaxis_title='Time [s]')

df2 = df[(df["Cycle [-]"] == 48) & (df["CRate [-]"] == 3)]
fig2 = px.scatter(df2, x='Time [s]', y="Voltage [V]", title="Plot of voltage during the last cycle measured at CRate 3")

app.layout = html.Div([
    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2),
    ]
)

In [5]:
app.run_server(mode="inline", port="5050")